<a href="https://colab.research.google.com/github/tateemma/Summarize-text-with-TextRank/blob/master/summarize_and_tweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import nltk
import tweepy
import time
nltk.download('punkt') # one time execution
import re
import chardet
nltk.download('stopwords') # one time execution
from nltk.corpus import stopwords
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize

In [ ]:
f = open('glove.6B.100d.txt', encoding='utf-8')
# GloVe word embeddings are vector representation of words and will be used to create vectors for sentences.
# extracting word vectors
word_embeddings = {}
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
#Twitter bot keys
CONSUMER_KEY = "your consumer key"
CONSUMER_SECRET = "your consumer secret"
ACCESS_KEY = "your access key"
ACCESS_SECRET = "your access secret"

In [ ]:
#Twitterbot authentication
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)
api = tweepy.API(auth)

In [ ]:
x = 0
for article in articles[x:]: #articles is the list of news articles we want to summarize
    print("SUMMARIZING ARTICLE :")
    sentences = []
    for s in article[1:]:
      sentences.append(sent_tokenize(s))

    sentences = [y for x in sentences for y in x] # flatten list

    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]

    stop_words = stopwords.words('english')

    # function to remove stopwords
    def remove_stopwords(sen):
        sen_new = " ".join([i for i in sen if i not in stop_words])
        return sen_new

    # remove stopwords from the sentences
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    # creating vectors for sentences by first fetching vectors 
    # (each of size 100 elements) for the constituent words in a sentence and then taking mean/average
    # of those vectors to arrive at a consolidated vector for the sentence.
    sentence_vectors = []
    
    for i in clean_sentences:
      if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
      else:
        v = np.zeros((100,))
      sentence_vectors.append(v)

    # similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])

    # initialize the matrix with cosine similarity scores
    for i in range(len(sentences)):
      for j in range(len(sentences)):
        if i != j:
          sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

    # applying TextRank algorithm to arrive at the sentence rankings
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)

    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    
    # Extract top sentences as the summary
    i = 0
    tweet = ("1. " + ranked_sentences[i][1] + " " + pagelinks[x])
    if len(tweet) > 280:
        i += 1
        tweet = ("1. " + ranked_sentences[i][1] + " " + pagelinks[x])
        
    print(tweet)
    api.update_status(tweet) #post on Twitter
    time.sleep(30)
    
    i += 1
    tweet = ("2. " + ranked_sentences[i][1])
    if len(tweet) > 280:
        i += 1
        tweet = ("2. " + ranked_sentences[i][1])
        
    print(tweet)
    api.update_status(tweet)#post on Twitter
    time.sleep(120)          
    print("\n")
    x += 1

SUMMARIZING ARTICLE :
1. “All went very well, players still finding their feet but the games were tightly contested with 3-2 wins,  “I would say it was an excellent start,” Mazviro told the Daily News. https://www.dailynews.co.zw//articles/2019/03/28/bva-league-kicks-off
2. Bulawayo Poly, on two points courtesy of their 3-2 win over Nust Vikings, are in third place while Vikings who are yet to pick up a point after two matches anchor the bottom of the table.


SUMMARIZING ARTICLE :
1. I should also take this opportunity to thank Iron Lion Nutrition who are also partnering almost every other competition, the federation is hosting. https://www.dailynews.co.zw//articles/2019/03/28/nfzbbf-praises-iron-lion-nutrition
2. “It is important also to note that even junior athletes are not at the same stage of development.


SUMMARIZING ARTICLE :
1. I think all the new players are doing good and should add what we were missing last season.” https://www.dailynews.co.zw//articles/2019/03/28/we-are-r



SUMMARIZING ARTICLE :
1. The good thing is that we are playing at home and we are also counting on our home fans to come in their numbers and rally behind us. https://www.dailynews.co.zw//articles/2019/03/23/zim-bank-on-home-support
2. I hope fans will also play their part, come in their numbers and support the boys.”


SUMMARIZING ARTICLE :
1. “I’m really excited that I managed to retain my good form regardless of injury, placing third on return always feels great,” Lynnette told the Daily News. https://www.dailynews.co.zw//articles/2019/03/22/mom-daughter-shine-at-cbz-marathon
2. “It was even a greater feeling to see my mother and I getting podium placements, there’s no better feeling for a daughter than learning from her, she’s the best mother on earth.


SUMMARIZING ARTICLE :
1. “Our partnership with Fidelity Life Assurance will provide us with medical solutions that will allow the game of football to keep moving forward. https://www.dailynews.co.zw//articles/2019/03/22/psl-fidel